# Выбор локации для скважины

Нужно решить, где бурить новую скважину.

Ниже предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Нужно проанализировать возможную прибыль и риски техникой *Bootstrap*.

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

импорт необходимых библиотек

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from numpy.random import RandomState
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
df_1 = pd.read_csv('C:/Users/Nshan/Desktop/Нефть_DS/geo_data_0.csv')
df_2 = pd.read_csv('C:/Users/Nshan/Desktop/Нефть_DS/geo_data_1.csv')
df_3 = pd.read_csv('C:/Users/Nshan/Desktop/Нефть_DS/geo_data_2.csv')

In [3]:
df_1.head(10)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


In [4]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
df_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [6]:
df_1.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [7]:
df_1.duplicated().sum()

0

Первая таблица содержит в себе 3 признака. В дальнейшем необходимо будет проанализировать возможность нормализации данных, чтобы каждый из параметров имел одинаковый вес.
Пропусков и дубликатов не обнаружено в первой таблице.

In [8]:
df_2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [9]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [10]:
df_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [11]:
df_2.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [12]:
df_2.duplicated().sum()

0

Во втором датафрейме пропусков и дубликатов нет. В дальнейшем необходимо будет нормализовать признаки данных.

In [13]:
df_3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [14]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [15]:
df_3.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [16]:
df_3.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [17]:
df_3.duplicated().sum()

0

В 3 датафрейме пропусков и дубликатов нет. В дальнейшем необходимо будет нормализовать признаки данных.

Поскольку столбец с id не является признаком, вляиющим на добычу нефти, то целессобразно удалить его.

In [18]:
df_1 = df_1.drop('id', 1)
df_2 = df_2.drop('id', 1)
df_3 = df_3.drop('id', 1)

**Выделим целевой признак и разделим данные на обучающую и валидационную**

In [19]:
def main_feature(df):
    features = df.drop('product', 1)
    target = df['product']
    return features, target

In [20]:
features_1, target_1 = main_feature(df_1)
features_2, target_2 = main_feature(df_2)
features_3, target_3 = main_feature(df_3)
features_1.head(10)

,f0,f1,f2
0,0.705745,-0.497823,1.221170
1,1.334711,-0.340164,4.365080
2,1.022732,0.151990,1.419926
3,-0.032172,0.139033,2.978566
4,1.988431,0.155413,4.751769
5,0.969570,0.489775,-0.735383
6,0.645075,0.530656,1.780266
7,-0.400648,0.808337,-5.624670
8,0.643105,-0.551583,2.372141
9,2.173381,0.563698,9.441852


Нормализация данных

In [21]:
def train_valid(features, target):
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
    return features_train, features_valid, target_train, target_valid

In [22]:
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_valid(features_1, target_1)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_valid(features_2, target_2)
features_train_3, features_valid_3, target_train_3, target_valid_3 = train_valid(features_3, target_3)
features_train_1.shape[0]

75000

In [23]:
def scale(features, change):
    scaler = StandardScaler()
    scaler.fit(features)
    features_sc = pd.DataFrame(scaler.transform(change), columns = features.columns)
    return features_sc

In [24]:
features_train_sc_1 = scale(features_train_1, features_train_1)
features_valid_sc_1 = scale(features_train_1, features_valid_1)
features_train_sc_2 = scale(features_train_2, features_train_2)
features_valid_sc_2 = scale(features_train_2, features_valid_2)
features_train_sc_3 = scale(features_train_3, features_train_3)
features_valid_sc_3 = scale(features_train_3, features_valid_3)

**Выводы**

1. Первая таблица содержит в себе 3 признака (столбец с id был удален, так как данный признак не влияет на целевой признак. 
2. Данный стандартизированы.
3. Пропусков и дубликатов в датафреймах не обнаружено.

## Обучение и проверка модели

Для обучения модели подходит только линейная регрессия, так как остальные — недостаточно предсказуемые.

In [25]:
def ml_learn(features_train, target_train, features_valid, target_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    result = mean_squared_error(predictions, target_valid)

    return predictions, result ** 0.5

In [26]:
print('Регион 1')
predictions_1, rmse =ml_learn(features_train_sc_1, target_train_1, features_valid_sc_1, target_valid_1)
print('Средний запас прдсказанного сырья в регионе:', predictions_1.mean())
print('Средне квадратическая ошибка составляет:', rmse)

Регион 1
Средний запас прдсказанного сырья в регионе: 92.59256778438035
Средне квадратическая ошибка составляет: 37.5794217150813


На примере первой модели можно сделать вывод, что модель с предсказанием справляется не точно, так как средне квадратическая ошибка (отклонение) составляет более 30% от средне предсказанной добычи в регионе.

In [27]:
print('Регион 2')
predictions_2, rmse =ml_learn(features_train_sc_2, target_train_2, features_valid_sc_2, target_valid_2)
print('Средний запас прдсказанного сырья в регионе:', predictions_2.mean())
print('Средне квадратическая ошибка составляет:', rmse)

Регион 2
Средний запас прдсказанного сырья в регионе: 68.728546895446
Средне квадратическая ошибка составляет: 0.8930992867756167


Вторая модель справляется неплохо с предсказанием, но при этом средняя предсказанная добыча во втором регионе в среднем на 28 % меньше чем в регионах 1 и 3.

In [28]:
print('Регион 3')
predictions_3, rmse =ml_learn(features_train_sc_3, target_train_3, features_valid_sc_3, target_valid_3)
print('Средний запас прдсказанного сырья в регионе:', predictions_3.mean())
print('Средне квадратическая ошибка составляет:', rmse)

Регион 3
Средний запас прдсказанного сырья в регионе: 94.96504596800489
Средне квадратическая ошибка составляет: 40.02970873393434


На примере 3 модели можно сделать вывод, что модель с предсказанием справляется не точно, так как средне квадратическая ошибка (отклонение) составляет более 30% от средне предсказанной добычи в регионе.

## Подготовка к расчёту прибыли

In [29]:
budget_all = 10000000000
budget_one = budget_all / 200 #для одной скважины в одном регионе
income_per_barrel = 450000
lim_research = 500
lim_invest = 200
print('Достаточный объём сырья для безубыточности новой скважины =', budget_one / income_per_barrel)

Достаточный объём сырья для безубыточности новой скважины = 111.11111111111111


Необходимый объём сырья для безубыточности новой скважины больше средних запасов в каждом из 3 регионов. разница особо ощутима для региона номер 2.

## Расчёт прибыли и рисков 

In [30]:
def profit(predictions, target_valid, lim_invest):
    predictions = pd.Series(predictions)
    target_valid = target_valid.reset_index(drop=True)
    predictions_sort = predictions.sort_values(ascending=False)
    revenue = target_valid[predictions_sort.index][:lim_invest]
    return revenue.sum() * income_per_barrel - budget_all

In [31]:
profit_1 = profit(predictions_1, target_valid_1, lim_invest)
profit_1

3320826043.1398506

In [32]:
profit_2 = profit(predictions_2, target_valid_2, lim_invest)
profit_2

2415086696.681511

In [33]:
profit_3 = profit(predictions_3, target_valid_3, lim_invest)
profit_3

2710349963.5998325

Наиболее прибыльны скважины в регионе номер 1.

In [34]:
state = RandomState(12345)
bootstrap_samples = 1000
def mean_prof_interval_risk_check(predictions, target_valid):
    values = []
    predictions = pd.Series(predictions)
    for i in range(bootstrap_samples):
        probabilities = predictions.sample(n=500, random_state=state, replace=True)
        value = profit(probabilities, target_valid, 200)
        values.append(value)
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    return values.mean(), lower, upper, (values < 0).mean()
    

In [35]:
profits_mean_1, lower, upper, risks = mean_prof_interval_risk_check(predictions_1, target_valid_1)

print('Средняя прибыль:', profits_mean_1)
print(f'Доверительный интервал 95% находится в промежутке от {lower} до {upper}')
print('Риск убытков в долях равен:', risks)

Средняя прибыль: 396164984.802371
Доверительный интервал 95% находится в промежутке от -111215545.89049526 до 909766941.5534226
Риск убытков в долях равен: 0.069


In [36]:
profits_mean_2, lower, upper, risks = mean_prof_interval_risk_check(predictions_2, target_valid_2)

print('Средняя прибыль:', profits_mean_2)
print(f'Доверительный интервал 95% находится в промежутке от {lower} до {upper}')
print('Риск убытков в долях равен:', risks)

Средняя прибыль: 461155817.27723986
Доверительный интервал 95% находится в промежутке от 78050810.7517417 до 862952060.2637234
Риск убытков в долях равен: 0.007


In [37]:
profits_mean_3, lower, upper, risks = mean_prof_interval_risk_check(predictions_3, target_valid_3)

print('Средняя прибыль:', profits_mean_3)
print(f'Доверительный интервал 95% находится в промежутке от {lower} до {upper}')
print('Риск убытков в долях равен:', risks)

Средняя прибыль: 392950475.17060375
Доверительный интервал 95% находится в промежутке от -112227625.37857565 до 934562914.5511636
Риск убытков в долях равен: 0.065


**Вывод**

Наиболее оптимальным для инвестирования в бурение новых скважин является регион номер два. Он оказался прибыльным и наименее рискованным.

## Общий вывод

1. Первая таблица содержит в себе 3 признака (столбец с id был удален, так как данный признак не влияет на целевой признак. 
2. Данный стандартизированы.
3. Пропусков и дубликатов в датафреймах не обнаружено.
4. Средний запас прдсказанного сырья в регионе 1: 92.59256778438035
5. Средне квадратическая ошибка составляет: 37.5794217150813
6. На примере первой модели можно сделать вывод, что модель с предсказанием справляется не точно, так как средне квадратическая ошибка (отклонение) составляет более 30% от средне предсказанной добычи в регионе
7. Средний запас прдсказанного сырья в регионе 2: 68.72854689544602
8. Средне квадратическая ошибка составляет: 0.893099286775617
9. Вторая модель справляется неплохо с предсказанием, но при этом средняя предсказанная добыча во втором регионе в среднем на 28 % меньше чем в регионах 1 и 3.
10. Средний запас прдсказанного сырья в регионе 3: 94.96504596800489
11. Средне квадратическая ошибка составляет: 0.893099286775617
12. На примере 3 модели можно сделать вывод, что модель с предсказанием справляется не точно, так как средне квадратическая ошибка (отклонение) составляет более 30% от средне предсказанной добычи в регионе.
13. Достаточный объём сырья для безубыточности новой скважины = 111.11111111111111
14. Необходимый объём сырья для безубыточности новой скважины больше средних запасов в каждом из 3 регионов. разница особо ощутима для региона номер 2.
15. Наиболее оптимальным для инвестирования в бурение новых скважин является регион номер один. Он оказался прибыльным и наименее рискованным.